<a href="https://colab.research.google.com/github/Salimgereyevm/SalimgereyevM309/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/Word2Vec/train.csv", index_col=0)
df.head()

,review,label
0,I think they really let the quality of the DVD...,0
1,I'm sorry but this is just awful. I have told ...,0
2,"The Japenese sense of pacing, editing and musi...",0
3,"In the '60's/'70's, David Jason was renowned f...",1
4,"""Hail The Woman"" is one of the most moving fil...",1


In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

en_stop = list(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 

def tokenize(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [t for t in tokens if
              re.match(r'[^\W\d]*$', t) and (len(t) > 2) and (t not in en_stop)]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

tokens = df['review'].apply(tokenize) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from gensim.models import Word2Vec
model = Word2Vec(tokens, size=300, window=6, min_count=4, iter=100, sg = 0, sample = 1e-5) 

In [0]:
from scipy import stats

def encode(list_of_tokens):
    x = np.array([model.wv[t] for t in list_of_tokens if t in model.wv.vocab])
    u = [np.mean(x, axis=0), np.median(x, axis = 0)]
    return np.concatenate(u)
fts = np.array([encode(t) for t in tokens])
fts.shape

(40000, 600)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(fts, df.label.values,
                                                    test_size=0.2, shuffle=True)

In [0]:
from sklearn import svm
clf1 = svm.SVC().fit(fts, df.label.values)

In [0]:
from sklearn.metrics import classification_report

# predicts = clf1.predict(X_train)
# print('Train\n', classification_report(y_train, predicts))

predicts = clf1.predict(X_test)
print('Test\n', classification_report(y_test, predicts, digits = 4))

Test
               precision    recall  f1-score   support

           0     0.9276    0.9154    0.9215      4056
           1     0.9142    0.9265    0.9203      3944

    accuracy                         0.9209      8000
   macro avg     0.9209    0.9210    0.9209      8000
weighted avg     0.9210    0.9209    0.9209      8000



In [0]:
test = pd.read_csv('/content/drive/My Drive/Word2Vec/test.csv',index_col = 0)

In [0]:
tok = test['review'].apply(tokenize)
mm = np.array([encode(t) for t in tok])
predicted = clf1.predict(mm)
pd.DataFrame({'Predicted' : predicted}).to_csv('/content/drive/My Drive/Word2Vec/solutionsvc.csv', index_label = 'Id')